In [1]:
import os
import pandas as pd
import numpy as np
from GAE.custom_layers import ConvTypes
from GAE.LossManager import LossTypes, LossManager
import scikit_posthocs as sp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from GAE.utils import *
import operator
import pickle 
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec
import re 

from FramsticksLib import *
import subprocess
from scipy.spatial import distance

DATA_OUT = 'plots/'

In [2]:
# experiments mask
path_models = "/home/adam/thesis/VGAE/experiments_new/models/models/"
path_configs = "/home/adam/thesis/VGAE/experiments_new/models/configs/"
model_names = "mask_all"


## multi_layers for gcs
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_multi_layers/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_multi_layers/models/No/configs/"
# model_names = "gcs_diff_layers"

## base_all 
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/No/configs/"
# model_names = "base_all_convtypes"

## joints
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/joints/configs/"
# model_names = "base_joints"


#### LOCALITY LOSS:
#all
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/configs_all/"
# model_names = "locality_all_convtypes"


#parts
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/joints/configs/"
# model_names = "locality_joints_convtypes"

#fitness
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/fitness/configs/"
# model_names = "locality_fitness_convtypes"

#dissim 
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_base/models/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_base/models/dissim/configs/"
# model_names = "locality_dissim_convtypes"

##VERTPOS
# path_models = "/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/"
# path_configs = "/home/adam/thesis/VGAE/experiments/experiment_vertpos/models_vertpos/configs/"
# model_names = "vertpos_all"


In [3]:
pathframs = "/home/adam/Framsticks/Framsticks50rc19"
framsmanager = FramsManager(pathframs)

framsLib = FramsticksLib(pathframs,None,["/home/adam/thesis/VGAE/framspy/eval-allcriteria_new.sim"])


Using Framsticks version: 5.0
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

Using Framsticks version: 5.0
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

Available objects: ['CheckpointEvent', 'Collision', 'CrCollision', 'Creature', 'CreatureSettings', 'CreatureSignals', 'CreatureSnapshot', 'Dictionary', 'ExpProperties', 'ExpState', 'ExtValue', 'File', 'FunctionReference', 'GenMan', 'GenManStats', 'GenePool', 'GenePools', 'Geno', 'Genotype', 'Interface', 'Joint', 'Loader', 'Math', 'MechJoint', 'MechPart', 'MessageCatcher', 'Model', 'ModelGeometry', 'ModelSymmetry', 'Neuro', 'NeuroClass', 'NeuroClassLibrary', 'NeuroDef', 'NeuroSignals', 'NeuronsSimEnabled', 'ODE', 'Orient', 'Part', 'Population', 'Populations', 'Ref', 'Signal', 'SignalView', 'SimilMeasure', 'SimilMeasureDistribution', 'SimilMeasureGreed

In [4]:
def get_convType(name):
    if name =="gcnconv":
        return "GCNConv"
    elif name =="armaconv":
        return "ARMAConv"
    elif name =="eccconv":
        return "ECCConv"
    elif name =="gatconv":
        return "GATConv"
    elif name =="gcsconv":
        return "GCSConv"
    
def get_Loss(name):
    if name =="joints":
        return "joints"
    elif name =="parts":
        return "parts"
    elif name =="fitness":
        return "fitness"
    elif name =="dissim":
        return "dissim"
    elif name =="None":
        return "No"

In [8]:
def load_losses(path):
    losses_all_test = []
    losses_all_train = []
    try:
        with open("{0}/losses_test.npy".format(path), "rb") as infile: 
            losses_all_test = np.load(infile).tolist()
        with open("{0}/losses_train.npy".format(path), "rb") as infile: 
            losses_all_train = np.load(infile).tolist()
#         print("Losses loaded successfully")
    except Exception as e:
        print("Error while loading Losses ", e)
    return losses_all_train, losses_all_test

In [11]:

columns = ["path","name","counter","loss_type","ae_type","convtype","latentdim","nhidden","convenc","denseenc","densedeca","convdecx","densedecx","path_model","losses_all_train", "losses_all_test","loss_train","loss_test"]
all_models= pd.DataFrame(columns=columns )
batchsize = 256
learningrate = 0.01
numfeatures = 4
adjsize = 15
counter = 0
for path, subdirs, files in os.walk(path_configs):
    for file_name in files:
        if "vTrue" in file_name:
            ae_type = "VGAE"
        else:
            ae_type = "GAE"
        splitted = file_name.split("_")
        
        convtype = get_convType(splitted[1])
        latentdim = splitted[2]
        nhidden = splitted[3]
        convenc = splitted[4]
        denseenc = splitted[5]
        densedeca = splitted[6]
        convdecx = splitted[7]
        densedecx = splitted[8]
        loss_type = get_Loss(splitted[9])
        

        for trainid in range(1,2,1):
            path_base = (
                loss_type+
                "/"+ae_type +
                "/numfeatures"+str(numfeatures) +
                "/adjsize"+str(adjsize) + 
                "/batchsize"+str(batchsize) +
                "/latentdim"+str(latentdim)+
                "/nhidden"+str(nhidden)+
                "/learningrate"+str(learningrate)+
                "/convtype"+str(convtype)+
                "/model_enc_"+str(convenc)+"_"+str(denseenc)+
                 "_deca"+str(densedeca)+
                 "_decx"+str(convdecx)+"_"+str(densedecx)+
                 "_train_id_"+str(trainid)
                )
            path_model = str(path_models)+path_base
            
            path_evol = path_model+"/0.333/0/"
            try:
                logs_path = path_evol+"logs.csv"
#                 print(logs_path)
                logs = pd.read_csv(path_evol+"logs.csv")
                print(max(logs['max']))
            except:
                print("No loss")
            losses_all_train, losses_all_test = load_losses(path_model)
            loss_train = losses_all_train[-1][1]
            loss_test = losses_all_test[-1][1]
            name = file_name.split("_")
#             print(name)
#             break
            f = operator.itemgetter(4,5,6,7,8)
            sublist = list(f(name))
#             print(sublist)
#             new_name = "d:{0} c:{1}".format(sublist[0],sublist[1])
#             new_name = "{0}{1}".format(sublist[0],sublist[1])
            new_name =""
            for e in sublist:
                new_name = new_name+e+"_" 
            name =new_name + ae_type+"_"+loss_type+"_"+latentdim
            data_model = [[path_base,name,counter,loss_type,ae_type,convtype,latentdim,nhidden,convenc,denseenc,densedeca,convdecx,densedecx,path_model,losses_all_train, losses_all_test,loss_train,loss_test]]
            data_model = pd.DataFrame(data_model,columns=columns) 
            all_models = all_models.append(data_model, ignore_index=True)
        counter+=1


No loss
No loss
No loss
0.5264425134172301
0.0156509178717988
-0.0100000000000004
No loss
-0.0100000000000004
0.5370429727033668
No loss
No loss
No loss
No loss
No loss
No loss
0.2521992216173244
No loss
No loss
-0.0100000000000004
0.381896475099256
0.4649081991364363
-0.0100000000000004
No loss
0.5796685859120465
-0.0100000000000004
No loss
No loss
No loss
-0.0100000000000004
0.6927227757815311
0.1386603609691102
0.3884969833532774
